In [1]:
import os
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
print("Num GPUs Available: ", tf.config.list_physical_devices())

Num GPUs Available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 1. No outlier treatment
Modelos entrenados sin tratamiento de outliers, hay dos casos:
- Datos normalizados entre 0 y 1
- Datos normalizados entre -1 y 1

Las imagenes en la carpeta plots se identifican de la siguiente forma:

{normalización}\_{variable}\_{neuronas}\_{ventana temporal}\_{epochs}ep_{batch size}bs.png

Por ejemplo: **01_T_64_24_15ep_32bs.png**
- Los datos son normalizados entre 0 y 1. 
- La variable que muestra la grafica es la Temperatura (T).
- La capa LSTM tiene 64 neuronas.
- La ventana temporal es de 24 (24 datos previos al instante predicho, cada una de estas 24 representa una medición cada 30min)
- Entrenado durante 15 epochs
- Batch size de 32.

In [13]:
station = 'C6.zip'

paths = [f'data/norm_data_01/{station}', f'data/norm_data_11/{station}']
input_width = [24, 48, 96]
neurons = [64]

for path in paths:
    df = pd.read_csv(path, compression='zip', header=0, sep=',')
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    df = df.astype({'T': 'float', 'HR': 'float', 'P': 'float', 'u2': 'float', 'v2': 'float', 'u6': 'float', 'v6': 'float', 'u10': 'float', 'v10': 'float', 'altitud': 'float', 'latitud': 'float', 'longitud': 'float'})
    
    min_maxs = json.load(open('data/min_maxs.json', 'r'))

    df['day'] = df['date'].dt.dayofyear / 365
    df['time'] = df['date'].dt.hour / 24

    df = df[['T', 'HR', 'P', 'u10', 'v10', 'day', 'time', 'date']]
    cols = ['T', 'HR', 'P', 'u10', 'v10']
    for width in input_width:
        prediction_width = 1

        df_train = df[df['date'] < '2019-01-01']
        df_test = df[df['date'] >= '2019-01-01']

        train_X = []
        train_Y = []

        for i in range(width, len(df_train) - prediction_width + 1):
            train_X.append(df_train.iloc[i - width:i][cols].values)
            train_Y.append(df_train.iloc[i:i + prediction_width][['T', 'u10', 'v10']].values)

        test_X = []
        test_Y = []

        for i in range(width, len(df_test) - prediction_width + 1):
            test_X.append(df_test.iloc[i - width:i][cols].values)
            test_Y.append(df_test.iloc[i:i + prediction_width][['T', 'u10', 'v10']].values)

        train_X = np.array(train_X)
        train_Y = np.array(train_Y)

        test_X = np.array(test_X)
        test_Y = np.array(test_Y)

        if path.split('/')[1].split('_')[-1][0] == '1':
            test_Y[:, 0, 0] = (test_Y[:, 0, 0] + 1) * ((min_maxs['T'][1] - min_maxs['T'][0]) * 0.5) + min_maxs['T'][0]
            test_Y[:, 0, 1] = (test_Y[:, 0, 1] + 1) * ((min_maxs['u10'][1] - min_maxs['u10'][0]) * 0.5) + min_maxs['u10'][0]
            test_Y[:, 0, 2] = (test_Y[:, 0, 2] + 1) * ((min_maxs['v10'][1] - min_maxs['v10'][0]) * 0.5) + min_maxs['v10'][0]
        else:
            test_Y[:, 0, 0] = test_Y[:, 0, 0] * (min_maxs['T'][1] - min_maxs['T'][0]) + min_maxs['T'][0]
            test_Y[:, 0, 1] = test_Y[:, 0, 1] * (min_maxs['u10'][1] - min_maxs['u10'][0]) + min_maxs['u10'][0]
            test_Y[:, 0, 2] = test_Y[:, 0, 2] * (min_maxs['v10'][1] - min_maxs['v10'][0]) + min_maxs['v10'][0]

        for n in neurons:
            model = Sequential()
            model.add(LSTM(n, activation='tanh', input_shape=(train_X.shape[1], train_X.shape[2])))
            model.add(Dropout(0.2))
            model.add(Dense(units=3, activation='linear'))

            model.compile(optimizer='adam', loss='mse', metrics=['mae'])
            print("Training model with input width: ", width, " and ", n, " neurons")
            with tf.device('/device:GPU:0'):
                history = model.fit(train_X, train_Y, epochs=15, batch_size=32, validation_split=0.1, verbose=1, shuffle=False)
            model.save(f'models/lstm_{path.split("_")[-1].split("/")[0]}_{n}_{width}_15epochs_32bs.h5')

            y_pred = model.predict(test_X)
            for idx, column in enumerate(['T', 'u10', 'v10']):
                plt.figure(figsize=(10, 6))

                if path.split('/')[1].split('_')[-1][0] == '1':
                    y_pred[:, idx] = (y_pred[:, idx] + 1) * ((min_maxs[column][1] - min_maxs[column][0]) * 0.5) + min_maxs[column][0]
                else:
                    y_pred[:, idx] = y_pred[:, idx] * (min_maxs[column][1] - min_maxs[column][0]) + min_maxs[column][0]
                
                plt.plot(test_Y[:1000, 0, idx], label='Real')
                plt.plot(y_pred[:1000, idx], label='Pred')
                
                rmse = np.sqrt(np.mean(np.power((test_Y[:, 0, idx] - y_pred[:, idx]), 2)))
                plt.legend()
                plt.title(f'{path.split("/")[1].split("_")[-1]}_{n}_{width}_15epochs_32bs{column}, RMSE: {rmse:.6f}')
                plt.savefig(f'plots/{path.split("/")[1].split("_")[-1]}_{column}_{n}_{width}_15ep_32bs.png')
                plt.clf()

545/545 [==============================] - 5s 7ms/step


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

# 2. Treated outliers

Modelos entrenados usando IQR como tratamiento de outliers, hay dos casos:
- Datos normalizados entre 0 y 1
- Datos normalizados entre -1 y 1

Las imagenes en la carpeta plots se identifican de la siguiente forma:

{normalización}\_{variable}\_{neuronas}\_{ventana temporal}\_{epochs}ep_{batch size}bs.png

Por ejemplo: **o01_T_64_24_15ep_32bs.png**
- Los datos son normalizados entre 0 y 1, la letra o indica que se han tratado outliers. 
- La variable que muestra la grafica es la Temperatura (T).
- La capa LSTM tiene 64 neuronas.
- La ventana temporal es de 24 (24 datos previos al instante predicho, cada una de estas 24 representa una medición cada 30min)
- Entrenado durante 15 epochs
- Batch size de 32.

In [14]:
station = 'C6.zip'

paths = [f'data/norm_data_o01/{station}']#, f'data/norm_data_o11/{station}'

input_width = [24, 48, 96]
neurons = [64]

for path in paths:
    df = pd.read_csv(path, compression='zip', header=0, sep=',')
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    df = df.astype({'T': 'float', 'HR': 'float', 'P': 'float', 'u2': 'float', 'v2': 'float', 'u6': 'float', 'v6': 'float', 'u10': 'float', 'v10': 'float', 'altitud': 'float', 'latitud': 'float', 'longitud': 'float'})
    
    min_maxs = json.load(open('data/quantiles.json', 'r'))

    df['day'] = df['date'].dt.dayofyear / 365
    df['time'] = df['date'].dt.hour / 24

    df = df[['T', 'HR', 'P', 'u10', 'v10', 'day', 'time', 'date']]
    cols = ['T', 'HR', 'P', 'u10', 'v10']
    for width in input_width:
        prediction_width = 1

        df_train = df[df['date'] < '2019-01-01']
        df_test = df[df['date'] >= '2019-01-01']

        train_X = []
        train_Y = []

        for i in range(width, len(df_train) - prediction_width + 1):
            train_X.append(df_train.iloc[i - width:i][cols].values)
            train_Y.append(df_train.iloc[i:i + prediction_width][['T', 'u10', 'v10']].values)

        test_X = []
        test_Y = []

        for i in range(width, len(df_test) - prediction_width + 1):
            test_X.append(df_test.iloc[i - width:i][cols].values)
            test_Y.append(df_test.iloc[i:i + prediction_width][['T', 'u10', 'v10']].values)

        train_X = np.array(train_X)
        train_Y = np.array(train_Y)

        test_X = np.array(test_X)
        test_Y = np.array(test_Y)

        if path.split('/')[1].split('_')[-1][1] == '1':
            test_Y[:, 0, 0] = (test_Y[:, 0, 0] + 1) * ((min_maxs['T'][1] - min_maxs['T'][0]) * 0.5) + min_maxs['T'][0]
            test_Y[:, 0, 1] = (test_Y[:, 0, 1] + 1) * ((min_maxs['u10'][1] - min_maxs['u10'][0]) * 0.5) + min_maxs['u10'][0]
            test_Y[:, 0, 2] = (test_Y[:, 0, 2] + 1) * ((min_maxs['v10'][1] - min_maxs['v10'][0]) * 0.5) + min_maxs['v10'][0]
        else:
            test_Y[:, 0, 0] = test_Y[:, 0, 0] * (min_maxs['T'][1] - min_maxs['T'][0]) + min_maxs['T'][0]
            test_Y[:, 0, 1] = test_Y[:, 0, 1] * (min_maxs['u10'][1] - min_maxs['u10'][0]) + min_maxs['u10'][0]
            test_Y[:, 0, 2] = test_Y[:, 0, 2] * (min_maxs['v10'][1] - min_maxs['v10'][0]) + min_maxs['v10'][0]

        for n in neurons:
            model = Sequential()
            model.add(LSTM(n, activation='tanh', input_shape=(train_X.shape[1], train_X.shape[2])))
            model.add(Dropout(0.2))
            model.add(Dense(units=3, activation='linear'))

            model.compile(optimizer='adam', loss='mse', metrics=['mae'])
            print("Training model with input width: ", width, " and ", n, " neurons")
            with tf.device('/device:GPU:0'):
                history = model.fit(train_X, train_Y, epochs=15, batch_size=32, validation_split=0.1, verbose=1, shuffle=False)
            model.save(f'models/lstm_{path.split("_")[-1].split("/")[0]}_{n}_{width}_15epochs_32bs.h5')
            
            y_pred = model.predict(test_X)
            for idx, column in enumerate(['T', 'u10', 'v10']):
                plt.figure(figsize=(10, 6))
                if path.split('/')[1].split('_')[-1][1] == '1':
                    y_pred[:, idx] = (y_pred[:, idx] + 1) * ((min_maxs[column][1] - min_maxs[column][0]) * 0.5) + min_maxs[column][0]
                else:
                    y_pred[:, idx] = y_pred[:, idx] * (min_maxs[column][1] - min_maxs[column][0]) + min_maxs[column][0]
                
                plt.plot(test_Y[:1000, 0, idx], label='Real')
                plt.plot(y_pred[:1000, idx], label='Pred')
                
                rmse = np.sqrt(np.mean(np.power((test_Y[:, 0, idx] - y_pred[:, idx]), 2)))
                plt.legend()
                plt.title(f'{path.split("/")[1].split("_")[-1]}_{n}_{width}_15epochs_32bs_{column}, RMSE: {rmse:.6f}')
                plt.savefig(f'plots/{path.split("/")[1].split("_")[-1]}_{column}_{n}_{width}_15ep_32bs.png')
                plt.clf()

545/545 [==============================] - 3s 4ms/step


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [15]:
%reset -f